In [223]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup

# Grabs the table from the html page,
# Creating a dataframe from the <table> element
r = requests.get("https://pokemondb.net/pokedex/all")
soup = BeautifulSoup(r.content, "html.parser")\
    .find("table")\
    .prettify()
pokedex = pd.read_html(soup)[0]
pokedex

,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,1,Bulbasaur,Grass Poison,318,45,49,49,65,65,45
1,2,Ivysaur,Grass Poison,405,60,62,63,80,80,60
2,3,Venusaur,Grass Poison,525,80,82,83,100,100,80
3,3,Venusaur Mega Venusaur,Grass Poison,625,80,100,123,122,120,80
4,4,Charmander,Fire,309,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...
1185,1004,Chi-Yu,Dark Fire,570,55,80,80,135,120,100
1186,1005,Roaring Moon,Dragon Dark,590,105,139,71,55,101,119
1187,1006,Iron Valiant,Fairy Fighting,590,74,130,90,120,60,116
1188,1007,Koraidon,Fighting Dragon,670,100,135,115,85,100,135


In [224]:
# Regexes to seperate pokemon type column into type 1 and type 2 columns
type_regex = r"([A-Za-z]+)\s?([A-Za-z]*)"

# New columns to store specific type information
pokedex['Type1'] = ''
pokedex['Type2'] = ''

# Converts the single type column into type1 and type2 columns
# If pokemon only has one type, then set type2 to None
for index, row in pokedex.iterrows():
    types = re.search(type_regex, row['Type'])
    if(types[0] == types[1]):
        pokedex.at[index, 'Type1'] = types[0]
        pokedex.at[index, 'Type2'] = None
    else:
        pokedex.at[index, 'Type1'] = types[0]
        pokedex.at[index, 'Type2'] = types[1]

# Removes the original single type column from the dataframe
pokedex = pokedex.drop(pokedex.columns[2], axis=1)

# Gets rid of duplicate pokemon that have a regional form or mega evolution
# This is so we can only compare the new pokemon of each generation, and not remakes
# Since this could lead to some confusion during comparisons
pokedex.drop_duplicates(subset='#', keep='first', inplace=True)
pokedex = pokedex.reset_index(drop=True)

pokedex

,#,Name,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Type1,Type2
0,1,Bulbasaur,318,45,49,49,65,65,45,Grass,Grass
1,2,Ivysaur,405,60,62,63,80,80,60,Grass,Grass
2,3,Venusaur,525,80,82,83,100,100,80,Grass,Grass
3,3,Venusaur Mega Venusaur,625,80,100,123,122,120,80,Grass,Grass
4,4,Charmander,309,39,52,43,60,50,65,Fire,None
...,...,...,...,...,...,...,...,...,...,...,...
1185,1004,Chi-Yu,570,55,80,80,135,120,100,Dark,Dark
1186,1005,Roaring Moon,590,105,139,71,55,101,119,Dragon,Dragon
1187,1006,Iron Valiant,590,74,130,90,120,60,116,Fairy,Fairy
1188,1007,Koraidon,670,100,135,115,85,100,135,Fighting,Fighting


In [ ]:
# Seperates each pokemon generation into its own dataframe by slicing the pokedex
# There wasn't an easier/more accurate way to do this with online databases,
# so I simply had to manually seperate the pokemon by their generations
gen1 = pokedex[0:151]
gen2 = pokedex[151:251]
gen3 = pokedex[251:386]
gen4 = pokedex[386:493]
gen5 = pokedex[493:649]
gen6 = pokedex[649:721]
gen7 = pokedex[721:809]
gen8 = pokedex[809:905]
gen9 = pokedex[905:1008]
# List that stores dataframe for each generation
generations = [gen1, gen2, gen3, gen4, gen5, gen6, gen7, gen8, gen9]
generations

In [247]:
# Here, we are creating a dataframe for the type matchups in pokemon
r = requests.get("https://pokemondb.net/type")
soup = BeautifulSoup(r.content, "html.parser")\
    .find("table")\
    .prettify()

type_matchups = pd.read_html(soup)[0]
type_matchups = type_matchups.rename(columns={type_matchups.columns[0]: 'Defense'})
# Replace all NaN values with 1, for the damage/defense multiplier would be 1
type_matchups = type_matchups.replace('½', .5)
type_matchups = type_matchups.replace(np.nan, 1)
type_matchups.astype('float64')
index = 1
type_matchups

KeyError: 2

In [244]:
# In order to compute the most powerful types, we are going to take the average of their
# type matchups.
print(type_matchups.dtypes)

Defense     object
Nor        float64
Fir         object
Wat         object
Ele         object
Gra         object
Ice         object
Fig         object
Poi         object
Gro         object
Fly         object
Psy         object
Bug         object
Roc         object
Gho         object
Dra         object
Dar         object
Ste         object
Fai         object
dtype: object
